# Jemné ladění modelů Open AI

Tento notebook vychází z aktuálních doporučení uvedených v dokumentaci [Fine Tuning](https://platform.openai.com/docs/guides/fine-tuning?WT.mc_id=academic-105485-koreyst) od Open AI.

Jemné ladění zlepšuje výkon základních modelů pro vaši aplikaci tím, že je znovu trénuje s dalšími daty a kontextem, které jsou relevantní pro konkrétní použití nebo scénář. Mějte na paměti, že techniky jako _few shot learning_ a _retrieval augmented generation_ umožňují vylepšit výchozí prompt o relevantní data a tím zvýšit kvalitu. Tyto přístupy jsou však omezeny maximální velikostí tokenového okna daného základního modelu.

Při jemném ladění vlastně znovu trénujeme samotný model s potřebnými daty (což nám umožňuje použít mnohem více příkladů, než kolik se vejde do maximálního tokenového okna) – a nasazujeme _vlastní_ verzi modelu, která už při vyhodnocování nepotřebuje příklady. To nejen zvyšuje efektivitu návrhu promptu (máme větší volnost v tom, jak tokenové okno využít), ale může také snížit náklady (protože při vyhodnocování posíláme modelu méně tokenů).

Jemné ladění má 4 kroky:
1. Připravte trénovací data a nahrajte je.
1. Spusťte trénovací úlohu a získejte jemně doladěný model.
1. Ověřte kvalitu jemně doladěného modelu a případně upravte.
1. Nasazujte jemně doladěný model pro vyhodnocování, když jste spokojeni.

Ne všechny základní modely podporují jemné ladění – [zkontrolujte dokumentaci OpenAI](https://platform.openai.com/docs/guides/fine-tuning/what-models-can-be-fine-tuned?WT.mc_id=academic-105485-koreyst) pro nejnovější informace. Jemně doladit můžete i model, který už byl jemně doladěn dříve. V tomto tutoriálu použijeme jako cílový základní model pro jemné ladění `gpt-35-turbo`.

---


### Krok 1.1: Připravte si svůj dataset

Pojďme vytvořit chatbot, který vám pomůže pochopit periodickou tabulku prvků tím, že bude odpovídat na otázky o prvku pomocí limeriku. V _tomto_ jednoduchém návodu jen vytvoříme dataset pro trénování modelu s několika ukázkovými příklady odpovědí, které ukazují očekávaný formát dat. V reálném použití byste potřebovali vytvořit dataset s mnohem více příklady. Můžete také využít otevřený dataset (pro vaši oblast použití), pokud existuje, a upravit ho pro použití při doladění.

Protože se zaměřujeme na `gpt-35-turbo` a chceme jednorázovou odpověď (chat completion), můžeme vytvořit příklady podle [tohoto doporučeného formátu](https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset?WT.mc_id=academic-105485-koreyst), který odpovídá požadavkům OpenAI na chat completion. Pokud očekáváte konverzaci na více tahů, použijete [formát pro více tahů](https://platform.openai.com/docs/guides/fine-tuning/multi-turn-chat-examples?WT.mc_id=academic-105485-koreyst), který obsahuje parametr `weight` pro určení, které zprávy mají být použity (nebo ne) při doladění.

Pro náš návod použijeme jednodušší formát pro jeden tah. Data jsou ve [formátu jsonl](https://jsonlines.org/?WT.mc_id=academic-105485-koreyst) s jedním záznamem na řádek, každý je reprezentován jako objekt ve formátu JSON. Ukázka níže ukazuje 2 záznamy jako příklad – kompletní vzorovou sadu (10 příkladů), kterou použijeme pro náš tutoriál doladění, najdete v [training-data.jsonl](../../../../../18-fine-tuning/python/openai/training-data.jsonl). **Poznámka:** Každý záznam _musí_ být definován na jednom řádku (nesmí být rozdělený na více řádků jako je běžné u formátovaného JSON souboru)

```json
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Gallium"}, {"role": "assistant", "content": "Gallium, oh gallium, so light - Melts in your hand, oh what a sight - At 86 degrees - Its liquid with ease - And in semiconductors, it's out of sight"}]}
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Hydrogen"}, {"role": "assistant", "content": "Hydrogen, the first in the line - The lightest of all, so divine - It's in water, you see - And in stars, it's the key - The universe's most common sign"}]}
```

V reálném použití budete potřebovat mnohem větší sadu příkladů pro dobré výsledky – je to kompromis mezi kvalitou odpovědí a časem/náklady na doladění. My používáme malou sadu, abychom mohli doladění rychle dokončit a ukázat postup. Podívejte se na [tento příklad v OpenAI Cookbook](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_finetune_chat_models.ipynb?WT.mc_id=academic-105485-koreyst) pro složitější návod na doladění.


### Krok 1.2 Nahrajte svůj dataset

Nahrajte data pomocí Files API [jak je popsáno zde](https://platform.openai.com/docs/guides/fine-tuning/upload-a-training-file). Upozorňujeme, že abyste mohli tento kód spustit, musíte nejprve provést následující kroky:
 - Nainstalovat Python balíček `openai` (ujistěte se, že používáte verzi >=0.28.0 kvůli nejnovějším funkcím)
 - Nastavit proměnnou prostředí `OPENAI_API_KEY` na váš OpenAI API klíč
Více informací najdete v [průvodci nastavením](./../../../00-course-setup/02-setup-local.md?WT.mc_id=academic-105485-koreyst), který je součástí kurzu.

Nyní spusťte kód, který vytvoří soubor pro nahrání z vašeho lokálního JSONL souboru.


In [24]:
from openai import OpenAI
client = OpenAI()

ft_file = client.files.create(
  file=open("./training-data.jsonl", "rb"),
  purpose="fine-tune"
)

print(ft_file)
print("Training File ID: " + ft_file.id)

FileObject(id='file-JdAJcagdOTG6ACNlFWzuzmyV', bytes=4021, created_at=1715566183, filename='training-data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)
Training File ID: file-JdAJcagdOTG6ACNlFWzuzmyV


### Krok 2.1: Vytvoření úlohy pro doladění pomocí SDK


In [25]:
from openai import OpenAI
client = OpenAI()

ft_filejob = client.fine_tuning.jobs.create(
  training_file=ft_file.id, 
  model="gpt-3.5-turbo"
)

print(ft_filejob)
print("Fine-tuning Job ID: " + ft_filejob.id)

FineTuningJob(id='ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', created_at=1715566184, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-EZ6ag0n0S6Zm8eV9BSWKmE6l', result_files=[], seed=830529052, status='validating_files', trained_tokens=None, training_file='file-JdAJcagdOTG6ACNlFWzuzmyV', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)
Fine-tuning Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh


### Krok 2.2: Zkontrolujte stav úlohy

Zde je několik věcí, které můžete dělat pomocí API `client.fine_tuning.jobs`:
- `client.fine_tuning.jobs.list(limit=<n>)` – Vypíše posledních n úloh pro doladění
- `client.fine_tuning.jobs.retrieve(<job_id>)` – Získá podrobnosti o konkrétní úloze pro doladění
- `client.fine_tuning.jobs.cancel(<job_id>)` – Zruší úlohu pro doladění
- `client.fine_tuning.jobs.list_events(fine_tuning_job_id=<job_id>, limit=<b>)` – Vypíše až n událostí z dané úlohy
- `client.fine_tuning.jobs.create(model="gpt-35-turbo", training_file="your-training-file.jsonl", ...)`

Prvním krokem tohoto procesu je _ověření trénovacího souboru_, abyste se ujistili, že data mají správný formát.


In [26]:
from openai import OpenAI
client = OpenAI()

# List 10 fine-tuning jobs
client.fine_tuning.jobs.list(limit=10)

# Retrieve the state of a fine-tune
client.fine_tuning.jobs.retrieve(ft_filejob.id)

# List up to 10 events from a fine-tuning job
client.fine_tuning.jobs.list_events(fine_tuning_job_id=ft_filejob.id, limit=10)

SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-GkWiDgZmOsuv4q5cSTEGscY6', created_at=1715566184, level='info', message='Validating training file: file-JdAJcagdOTG6ACNlFWzuzmyV', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-3899xdVTO3LN7Q7LkKLMJUnb', created_at=1715566184, level='info', message='Created fine-tuning job: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', object='fine_tuning.job.event', data={}, type='message')], object='list', has_more=False)

In [30]:
# Once the training data is validated
# Track the job status to see if it is running and when it is complete
from openai import OpenAI
client = OpenAI()

response = client.fine_tuning.jobs.retrieve(ft_filejob.id)

print("Job ID:", response.id)
print("Status:", response.status)
print("Trained Tokens:", response.trained_tokens)

Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh
Status: running
Trained Tokens: None


### Krok 2.3: Sledujte události pro monitorování postupu


In [44]:
# You can also track progress in a more granular way by checking for events
# Refresh this code till you get the `The job has successfully completed` message
response = client.fine_tuning.jobs.list_events(ft_filejob.id)

events = response.data
events.reverse()

for event in events:
    print(event.message)

Step 85/100: training loss=0.14
Step 86/100: training loss=0.00
Step 87/100: training loss=0.00
Step 88/100: training loss=0.07
Step 89/100: training loss=0.00
Step 90/100: training loss=0.00
Step 91/100: training loss=0.00
Step 92/100: training loss=0.00
Step 93/100: training loss=0.00
Step 94/100: training loss=0.00
Step 95/100: training loss=0.08
Step 96/100: training loss=0.05
Step 97/100: training loss=0.00
Step 98/100: training loss=0.00
Step 99/100: training loss=0.00
Step 100/100: training loss=0.00
Checkpoint created at step 80 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyyF2:ckpt-step-80
Checkpoint created at step 90 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyzhK:ckpt-step-90
New fine-tuned model created: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz
The job has successfully completed


### Krok 2.4: Zobrazit stav v OpenAI Dashboard


Stav můžete také zobrazit na webových stránkách OpenAI v sekci _Fine-tuning_ na platformě. Zde uvidíte stav aktuální úlohy a také můžete sledovat historii předchozích spuštění. Na tomto snímku obrazovky je vidět, že předchozí spuštění selhalo a druhý pokus byl úspěšný. Pro vysvětlení – k tomu došlo, když první spuštění použilo JSON soubor se špatně naformátovanými záznamy. Po opravě druhý pokus proběhl úspěšně a model byl zpřístupněn k použití.

![Fine-tuning job status](../../../../../translated_images/fine-tuned-model-status.563271727bf7bfba7e3f73a201f8712fae3cea1c08f7c7f12ca469c06d234122.cs.png)


Můžete si také prohlédnout stavové zprávy a metriky posunutím dolů ve vizuálním panelu, jak je znázorněno:

| Zprávy | Metriky |
|:---|:---|
| ![Zprávy](../../../../../translated_images/fine-tuned-messages-panel.4ed0c2da5ea1313b3a706a66f66bf5007c379cd9219cfb74cb30c0b04b90c4c8.cs.png) |  ![Metriky](../../../../../translated_images/fine-tuned-metrics-panel.700d7e4995a652299584ab181536a6cfb67691a897a518b6c7a2aa0a17f1a30d.cs.png)|


### Krok 3.1: Získání ID a otestování jemně doladěného modelu v kódu


In [46]:
# Retrieve the identity of the fine-tuned model once ready
response = client.fine_tuning.jobs.retrieve(ft_filejob.id)
fine_tuned_model_id = response.fine_tuned_model
print("Fine-tuned Model ID:", fine_tuned_model_id)

Fine-tuned Model ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz


In [47]:
# You can then use that model to generate completions from the SDK as shown
# Or you can load that model into the OpenAI Playground (in the UI) to validate it from there.
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model=fine_tuned_model_id,
  messages=[
    {"role": "system", "content": "You are Elle, a factual chatbot that answers questions about elements in the periodic table with a limerick"},
    {"role": "user", "content": "Tell me about Strontium"},
  ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content="Strontium, a metal so bright - It's in fireworks, a dazzling sight - It's in bones, you see - And in tea, it's the key - It's the fortieth, so pure, that's the right", role='assistant', function_call=None, tool_calls=None)


### Krok 3.2: Načtení a testování doladěného modelu v Playgroundu

Nyní můžete otestovat doladěný model dvěma způsoby. Nejprve můžete navštívit Playground a v rozbalovacím menu Models vybrat svůj nově doladěný model z nabízených možností. Druhou možností je použít volbu "Playground" zobrazenou v panelu Fine-tuning (viz screenshot výše), která spustí _porovnávací_ zobrazení, kde vedle sebe vidíte základní a doladěnou verzi modelu pro rychlé vyhodnocení.

![Fine-tuning job status](../../../../../translated_images/fine-tuned-playground-compare.56e06f0ad8922016497d39ced3d84ea296eec89073503f2bf346ec9718f913b5.cs.png)

Stačí vyplnit systémový kontext použitý ve vašich trénovacích datech a zadat testovací otázku. Všimnete si, že na obou stranách se aktualizuje stejný kontext i otázka. Spusťte porovnání a uvidíte rozdíl ve výstupech mezi oběma modely. _Všimněte si, jak doladěný model odpovídá ve formátu, který jste zadali ve svých příkladech, zatímco základní model pouze následuje systémový prompt_.

![Fine-tuning job status](../../../../../translated_images/fine-tuned-playground-launch.5a26495c983c6350c227e05700a47a89002d132949a56fa4ff37f266ebe997b2.cs.png)

Všimnete si také, že porovnání zobrazuje počet tokenů pro každý model a čas potřebný na inferenci. **Tento konkrétní příklad je zjednodušený a slouží pouze k ukázce postupu, nereprezentuje reálný dataset nebo scénář**. Můžete si všimnout, že oba vzorky mají stejný počet tokenů (systémový kontext a uživatelský prompt jsou totožné), přičemž doladěný model potřebuje na inferenci více času (vlastní model).

V reálných scénářích nebudete používat takto jednoduchý příklad, ale budete doladit model na reálných datech (například produktový katalog pro zákaznickou podporu), kde bude kvalita odpovědí mnohem zřetelnější. V _takovém_ případě bude dosažení srovnatelné kvality odpovědí se základním modelem vyžadovat více úprav promptu, což zvýší spotřebu tokenů a pravděpodobně i dobu zpracování inference. _Pokud si to chcete vyzkoušet, podívejte se na příklady doladění v OpenAI Cookbook._



---

**Prohlášení**:  
Tento dokument byl přeložen pomocí AI překladatelské služby [Co-op Translator](https://github.com/Azure/co-op-translator). Přestože se snažíme o přesnost, mějte prosím na paměti, že automatizované překlady mohou obsahovat chyby nebo nepřesnosti. Za autoritativní zdroj by měl být považován původní dokument v jeho rodném jazyce. Pro kritické informace doporučujeme profesionální lidský překlad. Neodpovídáme za žádná nedorozumění nebo nesprávné výklady vzniklé v důsledku použití tohoto překladu.
